In [4]:
!pip install requests beautifulsoup4 pandas matplotlib textblob langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=e2ebebfe8f6fbf96763ad3295ead4ceaab9db1cea37cc9de4e9c9376c5e7bca7
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [34]:
import requests
import pandas as pd
#from bs4 import BeautifulSoup
from langdetect import detect

app_id = "1446335066"
max_pages = 20  # Try fetching multiple pages (this may not always work)

all_reviews = []

for i in range(1, max_pages + 1):
    url = f"https://itunes.apple.com/us/rss/customerreviews/id={app_id}/sortBy=mostHelpful/page={i}/json"
    #url = f"https://itunes.apple.com/jp/rss/customerreviews/id={app_id}/sortBy=mostHelpful/page={i}/json" # For scraping Japan app
    #url = f"https://itunes.apple.com/in/rss/customerreviews/id={app_id}/sortBy=mostHelpful/page={i}/json" # For scraping India app

    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        break  # Stop if no more pages

    data = response.json()

    for entry in data.get("feed", {}).get("entry", [])[0:]:
        text = entry.get("content", {}).get("label", "")
        rating = entry.get("im:rating", {}).get("label", "")

        raw_date = entry.get("updated", {}).get("label", "")
        date = raw_date.split("T")[0] if "T" in raw_date else raw_date

        #print(f"Page {i}: {len(data.get('feed', {}).get('entry', [])) - 1} reviews")

        #try:
        #    if detect(text) == "en":
        #        all_reviews.append({"content": text, "rating": rating})
        #except:
        #    continue

        try:
          lang = detect(text)
        except:
          lang = "unknown"

        all_reviews.append({"content": text, "rating": rating, "date": date, "lang": lang})

df_apple_us = pd.DataFrame(all_reviews)
df_apple_us.to_csv("apple_us_reviews.csv", index=False, encoding="utf-8")
df_apple_us.head(5)

,content,rating,date,lang
0,"App works smooth, probably the number one fact...",5,2021-04-06,en
1,This app is pathetic! I consider myself to be ...,1,2021-07-03,en
2,This app is not helpful and simply isn’t worki...,1,2025-02-04,en
3,"If I could give it 0 stars I would, this app i...",1,2021-04-28,en
4,I don’t know yet how this is going to turn out...,1,2021-01-12,en
